In [ ]:
import numpy as np
from dataclasses import dataclass
from types import SimpleNamespace

### General Configs

In [2]:
@dataclass
class Params:
    
    # basic physical constants
    kref:       float = 1.0
    phi0:       float = 0.5
    lambdae:    float = 1.0
    lambdav:    float = 1.0
    mue:        float = 1.0
    muv:        float = 1.0
    delta:      float = 1.0
    k:          object = None   # constant/function/matrix, to be defined later

    # tc2 params
    Uref:       float = None
    Pref:       float = None
    Hv:         float = None
    Ha:         float = None
    length:     float = None
    omegax:     float = None
    omegat:     float = None

    # regularization constant
    lam:        float = 1e-5  # todo: what's this

    # grid coefficients
    nx:         int = 24
    nt:         int = 24
    deltax:     float = None
    deltat:     float = None
    x:          float = None
    h:          float = None

@dataclass
class Endpoints:
    xstart:     float = 0.0
    xend:       float = 2.0
    tstart:     float = 0.0
    tend:       float = 0.1

@dataclass
class Source:
    S:          object = None
    g:          object = None
    psi:        object = None
    F:          object = None
    bcu:        object = None
    bcp:        object = None
    IC:         object = None
    z3: object = None
    z4: object = None

| param              | meaning                                                   |
| ------------------ | --------------------------------------------------------- |
| `kref`             | permeability coefficient                                  |
| `phi0`             | porosity                                                  |
| `lambdae`, `mue`   | elastic Lamé constants (elastic region)                   |
| `lambdav`, `muv`   | viscoelastic Lamé constants (visco region)                |
| `delta`            | viscoelastic relaxation parameter                         |
| `Uref`, `Pref`     | reference displacement / pressure                         |
| `Ha`, `Hv`         | combined elastic/viscous moduli: Ha = λₑ+2μₑ, Hv = λᵥ+2μᵥ |
| `omegax`, `omegat` | spatial / temporal angular frequencies                    |
| `nx`, `nt`         | number of spatial / temporal divisions                    |
| `deltax`, `deltat` | step sizes                                                |
| `x`                | spatial grid array (linspace)                             |
| `length`           | domain length = xend - xstart                             |

| param     | meaning                                | form      |
| --------- | -------------------------------------- | --------- |
| `F(y,t)`  | control or forcing term                | λx,t: 0.0 |
| `S(y,t)`  | source term (body force)               | λy,t: ... |
| `g(t)`    | boundary condition (Dirichlet/Neumann) | λt: ...   |
| `psi(t)`  | another boundary/flux term             | λt: ...   |
| `z1`–`z4` | adjoint counterparts of above          | λt: 0.0   |
| `IC(y)`   | initial condition                      | λy: 0     |
| `bcu(y)`  | boundary u condition                   | λy: 0     |
| `bcp(y)`  | boundary p condition                   | λy: 0     |

### Helper Functions

In [3]:
def create_params_optimize():
    """
    Define parameters for PDE system. 
        1. initialize dataclasses Params() and Endpoints() from config
        2. create params for test cases
    """
    
    ### initialization
    params = Params()
    endpoints = Endpoints()
    source = Source()

    ### parameters for tc2 source functions
    params.Uref     = 1.0
    params.Pref     = 10.0
    params.Hv       = params.lambdav + 2 * params.muv
    params.Ha       = params.lambdae + 2 * params.mue
    params.length   = endpoints.xend - endpoints.xstart
    params.omegax   = 8 / params.length
    params.omegat   = 8 / (endpoints.tend - endpoints.tstart)

    return params, endpoints, source

In [4]:
def tc2source(params, xend):
    """
    Generate source and exact solution functions for Test Case 2 (TC2).

    Parameters
    ----------
    params : dataclass
        Must contain attributes:
        Uref, Pref, Ha, Hv, delta, kref, omegax, omegat, deltat.
    xend : float
        Right-endpoint of the spatial domain.

    Returns
    -------
    F, S, g, psi : callable
        Source and boundary term functions F(y,t), S(y,t), g(t), psi(t).
    exactu, exactp, exactut : callable
        Exact analytical solutions for u, p, and ∂u/∂t.
    """

    ### spatial and temporal test functions
    chi   = lambda y: np.sin(params.omegax * y)
    dchi  = lambda y: params.omegax * np.cos(params.omegax * y)
    d2chi = lambda y: -(params.omegax ** 2) * np.sin(params.omegax * y)

    tau  = lambda m: np.sin(params.omegat * m)
    dtau = lambda m: params.omegat * np.cos(params.omegat * m)

    ### define PDE source and boundary functions
    F = lambda y, t: -(params.Uref * d2chi(y) * 
        (params.Ha * tau(t) + params.delta * params.Hv * (tau(t) - tau(t - params.deltat)) / params.deltat)
        - params.Pref * tau(t) * dchi(y))

    S = lambda y, t: (params.Uref * ((tau(t) - tau(t - params.deltat)) / params.deltat) * 
        dchi(y) - params.kref * params.Pref * d2chi(y) * tau(t))

    g = lambda t: (params.Uref * dchi(xend) * (params.Ha * tau(t) + params.delta * params.Hv * (tau(t) - tau(t - params.deltat)) / params.deltat)
        - params.Pref * tau(t) * chi(xend))

    psi = lambda t: -params.kref * params.Pref * tau(t) * dchi(xend)

    ### exact solutions
    exactu  = lambda y, t: params.Uref * chi(y) * tau(t)
    exactp  = lambda y, t: params.Pref * chi(y) * tau(t)
    exactut = lambda y, t: params.Uref * chi(y) * dtau(t)

    return F, S, g, psi, exactu, exactp, exactut

In [5]:
def print_simplenamespace(matrices):

    for fname, val in matrices.__dict__.items():
        
        prefix = f'{fname}'
        if isinstance(val, np.ndarray):
            prefix += f', {val.shape} ='
        elif isinstance(val, list):
            prefix += f', list[{len(val)}] ='
        elif isinstance(val, (float, int, bool, np.number)):
            prefix += f', scalar ='
        else:
            prefix += f', type={type(val).__name__} ='
            
        print(prefix)
        print(val) 
        print('\n')
        
# def print_simplenamespace(ns, x_sample=None, t_sample=None, name="namespace"):
#     """
#     ns: SimpleNamespace / 任意有 __dict__ 的对象 (比如 source, matrices)
#     x_sample: 1D numpy array of x，用来测试 F(x,t)、S(x,t)、bcu(x)、bcp(x)
#     t_sample: 1D numpy array of t，用来测试 F(x,t)、S(x,t)、g(t)、psi(t)
#     """
#     print(f"\n========== {name} ==========\n")

#     for fname, val in ns.__dict__.items():
#         print(f"---- {fname} ----")

#         # 1. numpy array
#         if isinstance(val, np.ndarray):
#             print(f"type = ndarray, shape = {val.shape}")
#             # 打印前几行/前几个元素
#             if val.ndim == 1:
#                 print("values (first 5):", val[:5])
#             else:
#                 # 打印一个小切片
#                 sl = tuple(slice(0, min(3, s)) for s in val.shape)
#                 print("values (top-left small block):")
#                 print(val[sl])

#         # 2. 标量
#         elif isinstance(val, (float, int, bool, np.number)):
#             print("type = scalar")
#             print("value =", val)

#         # 3. list
#         elif isinstance(val, list):
#             print(f"type = list, len = {len(val)}")
#             if len(val) > 0:
#                 print("first element =", val[0])

#         # 4. 可调用对象（function / lambda 等）
#         elif callable(val):
#             print(f"type = function ({type(val).__name__})")
#             # 尝试在 sample 点上 evaluate
#             try:
#                 # 针对 F(x,t), S(x,t) 这种二元函数
#                 if fname in ("F", "S") and (x_sample is not None) and (t_sample is not None):
#                     # 用网格测试
#                     Xs, Ts = np.meshgrid(x_sample, t_sample)
#                     out = val(Xs, Ts)
#                     out = np.asarray(out)
#                     print(f"F/S evaluated on meshgrid -> shape = {out.shape}")
#                     sl = tuple(slice(0, min(3, s)) for s in out.shape)
#                     print("sample values (top-left):")
#                     print(out[sl])

#                 # g(t), psi(t) 这种一元函数（时间边界）
#                 elif fname in ("g", "psi") and (t_sample is not None):
#                     out = val(t_sample)
#                     out = np.asarray(out)
#                     print(f"g/psi evaluated on t_sample -> shape = {out.shape}")
#                     print("values (first 5):", out[:5])

#                 # bcu(y), bcp(y) 这种一元函数（空间边界）
#                 elif fname in ("bcu", "bcp") and (x_sample is not None):
#                     out = val(x_sample)
#                     out = np.asarray(out)
#                     print(f"bcu/bcp evaluated on x_sample -> shape = {out.shape}")
#                     print("values (first 5):", out[:5])

#                 else:
#                     # 通用兜底：如果有 x_sample，就测一个点；如果有 t_sample，就测一个点
#                     if (x_sample is not None) and (t_sample is not None):
#                         test_val = val(x_sample[0], t_sample[0])
#                         print("eval at (x_sample[0], t_sample[0]) ->", test_val)
#                     elif x_sample is not None:
#                         test_val = val(x_sample[0])
#                         print("eval at (x_sample[0]) ->", test_val)
#                     elif t_sample is not None:
#                         test_val = val(t_sample[0])
#                         print("eval at (t_sample[0]) ->", test_val)
#                     else:
#                         print("no x_sample/t_sample provided; only printing function repr.")
#             except Exception as e:
#                 print("  [eval failed]", e)

#         # 5. 其他类型（比如 struct-like / namespace / None / etc.）
#         else:
#             print("type =", type(val).__name__)
#             print(val)

#         print()   # 空行分隔

#     print("========== end ==========\n")

In [6]:
def algmatrices(params, setup, adjoint=False):
    """
    Builds element-level matrices Ai, B, C, D, G, MM, scriptB, scriptBuinv, Q, QQ, Lup, (Lhm).
    """

    ### 1. basis function integrals (explicit for linear elements)
    h = params.h
    aa = h / 3.0       # ∫ phi1^2
    ab = h / 6.0       # ∫ phi1*phi2
    bb = h / 3.0       # ∫ phi2^2

    ### 2. effective parameters
    params.mu = 2.0 * params.mue + 2.0 * params.delta * params.muv / params.deltat
    params.mp = 1.0 + params.delta * params.lambdav / (params.deltat * params.lambdae)
    mu = params.mu
    mp = params.mp

    ### 3. element-level base matrices
    Ai = (1.0 / (aa * bb - ab**2)) * np.array([[bb, -ab], [-ab, aa]])      # 2x2
    B = np.array([[-1.0, 1.0]])                             # 1x2
    C = np.array([[-1.0, 0.0],
                  [ 0.0, 1.0]])                             # 2x2
    D = np.array([[h/2.0],
                  [h/2.0]])                                 # 2x1
    G = np.array([[-1.0/h, 1.0/h]])                         # 1x2
    MM = Ai @ (mu * C + params.lambdae * mp * (D @ G))      # 2x2 element matrix for u-equation
    scriptB = float(B @ Ai @ B.T)        # 1x1 -> scalar
    scriptBuinv = 1.0 / (mu * scriptB)   # scaled inverse

    ### 4. Q and DQ
    Q = (1.0 / scriptB) * (B @ Ai @ C)   # 1x2
    Qvec = Q.reshape(1, -1)              # 1x2
    DQ = D @ Qvec                        # 2x2

    ### 5. QQ depends on forward/adjoint
    alpha = float(B @ Ai @ D)  # 1x1 → scalar
    # QQ = (scriptBuinv / params.deltat) * alpha * Q.T   # (2,1)
    if adjoint:
        return "adjoint"
    else:
        QQ = -scriptBuinv * (alpha * Q.T)

    QQ = np.asarray(QQ).reshape(2, 1)   # ensure 2x1
    QQvec = QQ.reshape(1, -1)           # 1x2

    ### 6. Lup
    # B.T : 2x1, QQvec : 1x2 -> 2x2, then Ai : 2x2
    Lup = -Ai @ (mu * (B.T @ QQvec))    # 2x2

    if adjoint:
        return "adjoint"
    else:
        Lup = Lup - Ai @ DQ
        Lhm = None

    # repeat Lup for all elements along columns
    Lup = np.tile(Lup, (1, params.nx))                # 2 x (2*nx)

    matrices = SimpleNamespace(
        Ai=Ai, B=B, C=C, D=D, G=G,
        MM=MM,
        scriptB=scriptB,
        scriptBuinv=scriptBuinv,
        Q=Q,
        Qvec=Qvec,
        DQ=DQ,
        QQ=QQ,
        Lup=Lup
    )
    if Lhm is not None:
        matrices.Lhm = Lhm

    return matrices, params

In [7]:
def algkmatrices(params, setup, matrices, k, adjoint=False):
    """
    Extends `matrices` with: scriptBpinv, R, bigMM, RR, Rvec, Luu, AiBR, Lpu, Lpp, (optionally Lhm in adjoint case).
    """

    ### 0. ensure k is at least 1D column vector
    k_arr = np.atleast_1d(np.array(k, dtype=float))     # shape (n_k,)
    k_col = k_arr.reshape(-1, 1)                        # (n_k,1)
    n_k = k_col.shape[0]
    B = matrices.B                                      # (1,2)
    Ai = matrices.Ai                                    # (2,2)
    C = matrices.C                                      # (2,2)
    D = matrices.D                                      # (2,1)
    G = matrices.G                                      # (1,2)
    MM = matrices.MM                                    # (2,2)
    Qvec = matrices.Qvec                                # (1,2)
    scriptB = float(matrices.scriptB)                   # scalar
    scriptBuinv = float(matrices.scriptBuinv)           # scalar

    ### 1. scriptBpinv and R
    scriptBpinv = 1.0 / (k_col * scriptB)               # (n_k,1)
    
    # Decide R according to setup and adjoint
    if (setup == 1) and (not adjoint):                  # constant linear, forward
        R = np.array([[0.0, 0.0]])                      # (1,2)
    elif (setup == 3) and (not adjoint):                # constant nonlinear, forward
        R = np.zeros((params.nx, 2))                    # (n,2)
    else:
        R = -params.h * (scriptBpinv @ G)
        if not adjoint:
            R = R / params.deltat

    ### 2. build bigAiDR and bigMM, then RR. n_R = R.shape[0] 
    R1_row = R[:, 0].reshape(1, -1, order="F")          # (1,n_R)
    R2_row = R[:, 1].reshape(1, -1, order="F")          # (1,n_R)
    AiDR1 = Ai @ (D @ R1_row)                           # (2, n_R)
    AiDR2 = Ai @ (D @ R2_row)                           # (2, n_R)

    # stack vertically to 4 x n_R, then reshape (2, 2*n_R) in column-major ("F")
    bigAiDR = np.reshape(np.vstack([AiDR1, AiDR2]), (2, -1), order="F")# (2, 2*n_R)
    c = bigAiDR.shape[1]                                # c = 2*n_R
    bigMM = np.tile(MM, (1, c // 2))                    # 2 x c. bigMM is MM repeated along columns to match c
    RR = scriptBuinv * (B @ bigMM)                      # (1, c)

    if not adjoint:
        RR = RR - scriptBuinv * (B @ bigAiDR)
    else:
        return "adjoint"

    ### 3. vectorize Luu and Lpu for PVE model
    Rvec = (R.T).reshape(1, -1, order="F")           # (1, 2*n_R)
    DR = D @ Rvec                                    # (2, 2*n_R)
    Luu1 = bigMM - Ai @ (params.mu * (B.T @ RR))     # (2, c)
    k_row = k_col.reshape(1, -1, order="F")          # (1, n_k)
    kjvec = np.repeat(k_row, 2, axis=1)              # (1, 2*n_k)

    if not adjoint:
        Luu = Luu1 - Ai @ DR                                # (2, 2*n_R)
        AiBR = Ai @ (B.T @ Rvec)                            # (2, 2*n_R), broadcast
        Lpu_local = AiBR * kjvec                            # (2, 2*n_R) if n_k == n_R
        Lpu = np.tile(Lpu_local, (1, 2 * params.nx // c))   # (2, 2*nx)
        matrices.AiBR = AiBR
        matrices.Lpu = Lpu
    else:
        return "adjoint"

    c_DR = DR.shape[1]                               # c_DR = 2*n_R = c
    Luu = np.tile(Luu, (1, 2*params.nx//c_DR))       # (2, 2*nx)

    ### 4. vectorize Lpp
    Lppnok = Ai @ C - Ai @ (B.T @ Qvec)              # (2,2)
    BigLppnok = np.tile(Lppnok, (1, n_k))            # (2, 2*n_k)
    Lpp_local = -BigLppnok * kjvec                   # (2, 2*n_k)
    Lpp = np.tile(Lpp_local, (1, 2*params.nx//c_DR)) # (2, 2*nx)

    # construct matrices
    matrices.scriptBpinv = scriptBpinv
    matrices.R = R
    matrices.bigAiDR = bigAiDR
    matrices.bigMM = bigMM
    matrices.RR = RR                                 # (1, c)
    matrices.Rvec = Rvec
    matrices.DR = DR
    matrices.Luu = Luu
    matrices.Lpp = Lpp
    return matrices

In [8]:
def algPivectors(params, setup, matrices, source, adjoint=False,
                 endpoints=None, t=None, Pi=None, hi=None):
    return "haven't checked this function yet"

    # midpoints of spatial grid
    x = np.asarray(params.x)
    x_input = 0.5 * x[:-1] + 0.5 * x[1:]      # (n,)

    # check which "branch" we are in
    if (-1)**setup == 1:
        # ---------- TIME-DEPENDENT / ADJOINT CASE ----------
        # 1) F at midpoints
        if callable(getattr(source, "F")):
            # F(x, t) function
            Fvec = source.F(x_input, t)       # shape (n,) or (n,1)
        else:
            # F stored as array. We interpret columns as time layers
            F_arr = np.asarray(source.F)
            # time index (MATLAB: round((t - tstart)/deltat))
            idx = int(round((t - endpoints.tstart) / params.deltat))

            if F_arr.ndim == 1:
                # 1D -> assume nodal, length n+1
                Fnod = F_arr
            else:
                # assume shape (spatial, time)
                Fnod = F_arr[:, idx]

            Fnod = np.asarray(Fnod)
            # if it's nodal (n+1), average to midpoints
            if Fnod.shape[0] == params.nx + 1:
                Fvec = 0.5 * Fnod[:-1] + 0.5 * Fnod[1:]
            else:
                # already element-based
                Fvec = Fnod

        Fvec = Fvec.reshape(params.nx, 1)      # (n,1)
        matrices.biplus1 = params.h * Fvec    # (n,1)

        # 2) S at midpoints
        if callable(getattr(source, "S")):
            Svec1 = source.S(x_input, t)
        else:
            S_arr = np.asarray(source.S)
            idx = int(round((t - endpoints.tstart) / params.deltat)) + 1
            if S_arr.ndim == 1:
                Snod = S_arr
            else:
                Snod = S_arr[:, idx]
            Snod = np.asarray(Snod)
            if Snod.shape[0] == params.nx + 1:
                Svec1 = 0.5 * Snod[:-1] + 0.5 * Snod[1:]
            else:
                Svec1 = Snod

        Svec = Svec1.reshape(params.nx, 1)     # (n,1)
        liplus1 = (params.h * Svec).T         # (1,n)

        # ---- handle liplus1 with Pi depending on how many args we have ----
        # emulate MATLAB:
        # if nargin==7 -> liplus1 - h/(lambdae*deltat)*Pi
        # elseif nargin~=4 && nargin~=6 -> -liplus1
        has_Pi = Pi             is not None
        has_adjoint = adjoint   is not None
        has_hi = hi             is not None

        if has_Pi and not has_adjoint and not has_hi:
            # MATLAB: nargin == 7
            Pi_row = np.atleast_1d(Pi).reshape(1, -1)
            matrices.liplus1 = liplus1 - (params.h / (params.lambdae * params.deltat)) * Pi_row
        elif (has_adjoint or has_hi) or (has_Pi and (has_adjoint or has_hi)):
            # MATLAB: nargin !=4 && !=6 (mainly 8、9)
            matrices.liplus1 = -liplus1
        else:
            # fallback: just store liplus1
            matrices.liplus1 = liplus1

        # ---- ri: coupling Pi/hi through Ai,D ----
        Ai = matrices.Ai    # (2,2)
        D  = matrices.D     # (2,1)

        # base term with Pi
        if has_Pi:
            Pi_row = np.atleast_1d(Pi).reshape(1, -1)    # (1,m)
            # Ai * D * Pi_row : (2,2)*(2,1)*(1,m) -> (2,m)
            AiD_Pi = Ai @ (D @ Pi_row)
            ri = (params.delta * params.Hv / (params.deltat * params.lambdae)) * AiD_Pi
        else:
            ri = np.zeros((2, Svec.shape[0]))  # fallback

        # extra correction with hi
        if has_hi:
            hi_row = np.atleast_1d(hi).reshape(1, -1)
            AiD_hi = Ai @ (D @ hi_row)         # (2,m)
            ri = -ri + (1.0 / params.deltat) * AiD_hi

        matrices.ri = ri                       # (2,m)

        # ---- fiplus1nok: combine with B*ri ----
        B = matrices.B                         # (1,2)

        fiplus1nok = matrices.biplus1          # (n,1)
        # B * ri : (1,2)*(2,m) -> (1,m), transpose -> (m,1)
        Bri = (B @ ri).T                       # (m,1)

        # MATLAB:
        # if nargin<8 -> + Bri; else -> - Bri
        if not has_adjoint and not has_hi:
            matrices.fiplus1nok = fiplus1nok + Bri
        else:
            matrices.fiplus1nok = fiplus1nok - Bri

        return matrices

    else:
        # ---------- TIME-INDEPENDENT / OTHER SETUPS ----------
        # F at midpoints (no t, no endpoints)
        if callable(getattr(source, "F")):
            Fvec = source.F(x_input)
        else:
            F_arr = np.asarray(source.F)
            if F_arr.ndim == 1 and F_arr.shape[0] == params.nx + 1:
                Fvec = 0.5 * F_arr[:-1] + 0.5 * F_arr[1:]
            else:
                Fvec = F_arr

        # S similarly
        if callable(getattr(source, "S")):
            Svec = source.S(x_input)
        else:
            S_arr = np.asarray(source.S)
            if S_arr.ndim == 1 and S_arr.shape[0] == params.nx + 1:
                Svec = 0.5 * S_arr[:-1] + 0.5 * S_arr[1:]
            else:
                Svec = S_arr

        Fvec = Fvec.reshape(-1, 1)              # (l,1)
        Svec = Svec.reshape(-1, 1)              # (l,1)

        matrices.biplus1 = params.h * Fvec
        matrices.liplus1 = (params.h * Svec).T  # (1,l)

        # setup==1 && nargin>5: take negative?
        if setup == 1 and (Pi is not None or adjoint is not None or hi is not None):
            matrices.liplus1 = -matrices.liplus1

        matrices.fiplus1nok = matrices.biplus1
        matrices.ri = np.zeros((2, len(matrices.fiplus1nok)))
        return matrices

In [9]:
def bmatrices(params, matrices):
    """
    Build time-stepping coupling matrices Bcal, BcalS, BcalF
    for the linear PVE model.

    This version handles the case where params.kref is scalar.
    """

    Ai = np.asarray(matrices.Ai, dtype=float)                # 2x2
    B  = np.asarray(matrices.B,  dtype=float).reshape(1, 2)  # 1x2
    D  = np.asarray(matrices.D,  dtype=float).reshape(2, 1)  # 2x1
    G  = np.asarray(matrices.G,  dtype=float).reshape(1, 2)  # 1x2

    h      = float(params.h)
    dt     = float(params.deltat)
    delta  = float(params.delta)
    Hv     = float(params.Hv)
    mu     = float(params.mu)
    nx     = int(params.nx)

    scriptB     = float(matrices.scriptB)
    scriptBuinv = float(matrices.scriptBuinv)
    scriptBpinv = float(matrices.scriptBpinv)

    sgn = (-1)**nx

    ### 1. ri_no_u, lu; ri_no_u = - delta * Hv / dt * Ai * D * G   (2x2)
    DG = D @ G                              # 2x2
    ri_no_u = - delta * Hv / dt * (Ai @ DG) # 2x2
    lu = (h / dt) * G                       # 1x2

    ### 2. fu, fqS
    Bri = B @ ri_no_u                          # 1x2
    tmp_scalar = float(B @ (Ai @ D))           # 1x1
    halfh = np.array([[h/2.0, h/2.0]])         # 1x2
    fu = Bri - tmp_scalar * scriptBpinv * lu   # 1x2
    fqS = - tmp_scalar * scriptBpinv * halfh   # 1x2

    ### 3. buu, bpu, betauS, betapS, betauF
    AiB = Ai @ B.T     # 2x1
    AiD = Ai @ D       # 2x1
    term2 = mu * scriptBuinv * (AiB @ fu)          # 2x2
    term3 = scriptBpinv * (AiD @ lu)               # 2x2
    buu = ri_no_u - term2 - term3                  # 2x2
    bpu = (1.0 / scriptB) * (AiB @ lu)             # 2x2

    # betauS = -mu*Ai*B'*scriptBuinv*fqS - Ai*D*scriptBpinv*halfh
    termS1 = mu * scriptBuinv * (AiB @ fqS)        # 2x2
    termS2 = scriptBpinv * (AiD @ halfh)           # 2x2
    betauS = - termS1 - termS2                     # 2x2
    
    betapS = (1.0 / scriptB) * (AiB @ halfh)       # 2x2
    betauF = - mu * scriptBuinv * (AiB @ halfh)    # 2x2

    ### 4. assemble global matrices Bcal, BcalS, BcalF
    dof = 2 * nx + 2
    Bcal  = np.zeros((dof, dof))
    BcalS = np.zeros((dof, nx+1))
    BcalF = np.zeros((dof, nx+1))

    offset = nx + 1   # p starting index

    # first pass: put local blocks on diagonal blocks
    for ix in range(1, int(np.ceil(nx/2))+1):
        if 2*ix <= nx + 1:
            i = 2*ix - 1
            j = 2*ix
            # Python 0-based slice
            iu = slice(i-1, j)                      # u-block row/col
            ip = slice(offset + i-1, offset + j)    # p-block row

            Bcal[iu, iu]  += sgn * buu
            BcalS[iu, iu] += sgn * betauS
            Bcal[ip, iu]  += sgn * bpu
            BcalS[ip, iu] += sgn * betapS
            BcalF[iu, iu] += sgn * betauF

    # second pass: subtract off overlapping contributions between neighboring elements
    for ix in range(1, int(np.ceil(nx/2))+1):
        if 2*ix + 1 <= nx + 1:
            i = 2*ix
            j = 2*ix + 1
            iu = slice(i-1, j)                      # [i, j] -> [i-1 : j]
            ip = slice(offset + i-1, offset + j)

            Bcal[iu, iu]  -= sgn * buu
            BcalS[iu, iu] -= sgn * betauS
            Bcal[ip, iu]  -= sgn * bpu
            BcalS[ip, iu] -= sgn * betapS
            BcalF[iu, iu] -= sgn * betauF

    # zero out boundary rows (Dirichlet boundary; BC handled separately)
    Bcal[0, :]        = 0.0
    Bcal[nx+1, :]     = 0.0
    BcalS[0, :]       = 0.0
    BcalS[nx+1, :]    = 0.0
    BcalF[0, :]       = 0.0
    BcalF[nx+1, :]    = 0.0

    matrices.ri_no_u = ri_no_u
    matrices.lu      = lu
    matrices.fu      = fu
    matrices.fqS     = fqS

    matrices.buu     = buu
    matrices.bpu     = bpu
    matrices.betauS  = betauS
    matrices.betapS  = betapS
    matrices.betauF  = betauF

    matrices.Bcal    = Bcal
    matrices.BcalS   = BcalS
    matrices.BcalF   = BcalF

    return matrices

In [ ]:
def derive_b(matrices, source, uhat, x, t, setup, endpoints, params, BC):
    """
    Build RHS vector b for the (u_hat, p_hat) linear system at time t.

    Parameters
    ----------
    matrices : SimpleNamespace / object with attributes
        Must contain Bcal, BcalS, BcalF.
    source   : object
        Must contain F, S, bcu, bcp, g, psi.
    uhat     : array, shape (n+1,)
        current step's displacement dof vector (u^k_hat).
    x        : array, shape (n+1,)
        spatial grid nodes.
    t        : float
        current time.
    setup    : int
        model setup index (1,2,3,4 ...).
    endpoints: object with attributes tstart
    params   : object with attributes n, nx, deltat
    BC       : object with attributes g, psi, bcu, bcp 
    """

    n = int(params.nx)
    dt = float(params.deltat)

    ### 1. construct Fvec, Svec
    if (-1)**setup == 1:
        # time-dependent branch (setup even)
        # F
        if callable(getattr(source, 'F', None)):
            Fvec = source.F(x, t)
        else:
            idx = int(round((t - endpoints.tstart) / dt))
            Fmat = np.asarray(source.F)
            Fvec = Fmat[:, idx]
        Fvec = np.asarray(Fvec).reshape(-1, 1)   # (n+1,1)

        # S
        if callable(getattr(source, 'S', None)):
            Svec = source.S(x, t)
        else:
            idxS = int(round((t - endpoints.tstart) / dt)) + 1
            Smat = np.asarray(source.S)
            Svec = Smat[:, idxS]
        Svec = np.asarray(Svec).reshape(-1, 1)   # (n+1,1)

    else:
        # time-independent branch
        if callable(getattr(source, 'F', None)):
            Fvec = source.F(x)
        else:
            Fvec = np.asarray(source.F)
        Fvec = Fvec.reshape(-1, 1)

        if callable(getattr(source, 'S', None)):
            Svec = source.S(x)
        else:
            Svec = np.asarray(source.S)
        Svec = Svec.reshape(-1, 1)

    ### 2. patch y = [uhat; 0]
    uhat = np.asarray(uhat).reshape(-1, 1)  # (n+1,1)
    zeros_block = np.zeros_like(uhat)       # (n+1,1)
    y = np.vstack([uhat, zeros_block])      # (2n+2,1)

    Bcal  = np.asarray(matrices.Bcal)
    BcalS = np.asarray(matrices.BcalS)
    BcalF = np.asarray(matrices.BcalF)

    b = Bcal @ y + BcalS @ Svec + BcalF @ Fvec    # (2n+2,1)

    ### 3. adding g, psi BC
    if (-1)**setup == 1:
        # time-dependent case: BC.g, BC.psi
        b[n, 0]       += BC.g      # b(params.n+1)
        b[2*n + 1, 0] += BC.psi    # b(2*params.n+2)
    else:
        # time-independent case: source.g, source.psi
        b[n, 0]       += source.g
        b[2*n + 1, 0] += source.psi

    ### 4. LHS Dirichlet BC u(0), p(0)
    # b(1)       = source.bcu(0)
    # b(n+2)     = source.bcp(0)
    b[0, 0]     = source.bcu(0.0)
    b[n+1, 0]   = source.bcp(0.0)

    return b

In [19]:
def solveuhatphat(Luu, Lup, Lpu, Lpp, b, n):
    """
    Assemble global matrix M from block-rows (Luu, Lup, Lpu, Lpp),
    apply Dirichlet BC on first u and first p, solve M v = b,
    and split v into uhat, phat.

    Parameters
    ----------
    Luu, Lup, Lpu, Lpp : ndarray
        Each of shape (2, 2*n): the k-th element block is in [:, 2*k:2*k+2].
    b : ndarray
        Right-hand side, shape (2*n+2,) or (2*n+2,1).
    n : int
        Number of elements.

    Returns
    -------
    uhat : ndarray, shape (n+1,)
    phat : ndarray, shape (n+1,)
    M    : ndarray, shape (2*n+2, 2*n+2)
    """

    # ensure arrays
    Luu = np.asarray(Luu, dtype=float)
    Lup = np.asarray(Lup, dtype=float)
    Lpu = np.asarray(Lpu, dtype=float)
    Lpp = np.asarray(Lpp, dtype=float)

    b = np.asarray(b, dtype=float).reshape(-1)  # (2n+2,)

    dof = 2*n + 2
    M = np.zeros((dof, dof), dtype=float)

    # assemble global matrix
    for k in range(n):       # Python: k = 0,...,n-1 对应 MATLAB: k=1,...,n
        sign = (-1)**(n - (k+1))   # 对应 MATLAB 里的 (-1)^(n-k)

        # element block columns in L.. (0-based)
        cols = slice(2*k, 2*k + 2)           # 2*k-1:2*k (MATLAB) -> 2*k:2*k+2 (Python)

        # global indices for u DOFs: k:k+1  (MATLAB 1-based)
        iu = slice(k, k+2)                   # rows/cols [k, k+1] in 0-based

        # global indices for p DOFs: k+n+1:k+n+2 (MATLAB)
        ip = slice(n + 1 + k, n + 1 + k + 2) # 0-based: [n+1+k-1, n+2+k-1] = [n+k, n+k+2)

        # extract local 2x2 blocks
        Luu_k = Luu[0:2, cols]
        Lup_k = Lup[0:2, cols]
        Lpu_k = Lpu[0:2, cols]
        Lpp_k = Lpp[0:2, cols]

        # add to global matrix
        M[iu, iu]       += sign * Luu_k
        M[iu, ip]       += sign * Lup_k
        M[ip, iu]       += sign * Lpu_k
        M[ip, ip]       += sign * Lpp_k

    # Dirichlet BC: u_0 = 0, p_0 = 0
    M[0, :]       = 0.0
    M[0, 0]       = 1.0

    M[n+1, :]     = 0.0
    M[n+1, n+1]   = 1.0

    # solve linear system
    nv = np.linalg.solve(M, b)   # (2n+2,)

    # split into uhat, phat
    uhat = nv[0:n+1]             # 0..n
    phat = nv[n+1:2*n+2]         # n+1..2n+1

    return uhat, phat, M

In [12]:
def constlinPVE(source, params, adjoint=False):
    """
    PDE
    ↓  discretization
    algmatrices
    ↓  insert parameter effects
    algkmatrices
    ↓  build source-term vector
    algPivectors
    ↓  finalize FEM load vector
    bmatrices
    ↓  assemble global matrix
    solveuhatphat
    ↓  apply BC + solve
    [ û , p̂ ]
    """

    return "not corresponding test cases so far"

    # matrices, params = algmatrices(params, 1, adjoint)
    # matrices = algkmatrices(params, 1, matrices, params.kref, adjoint)
    # matrices = algPivectors(params, 1, matrices, source, adjoint)

    # if adjoint:
    #     return "adjoint constlinPVE"
    # else:
    #     matrices = bmatrices(params, matrices)
    #     Luu, Lup, Lpu, Lpp = matrices.Luu, matrices.Lup, matrices.Lpu, matrices.Lpp
    #     rhs_m, rhs_h = matrices.buu, matrices.bpu

    # mhat, hhat, M, b = solveuhatphat(Luu, Lup, Lpu, Lpp, rhs_m, rhs_h, params.nx, source, matrices, params)
    # u, p = solveup(mhat, hhat, matrices)
    # return u, p, mhat, hhat, M, b, matrices

In [ ]:
def linPVE(source, params, endpoints, adjoint: bool = False, debug=False):
    """
    Linear (non-constant k) PVE solver.

    PDE in (x,t)
      ↓  space FE discretization
    algmatrices(setup=2)
      ↓  insert permeability k(x,t) effects
    algkmatrices(setup=2)
      ↓  (forward) assemble space–time load at each step
      ↓  (adjoint) assemble Pi-dependent RHS at each step
    bmatrices / derive_b / algPivectors / adjbvectors
      ↓  assemble global block system in time
    solveuhatphat
      ↓
      [û(x,t_k), p̂(x,t_k)] over all time steps
      ↓
    solveup
      ↓
    u(x,t), p(x,t)
    """

    ### spatial grid and midpoints
    nx, nt = params.nx, params.nt
    x = np.linspace(endpoints.xstart, endpoints.xend, nx + 1)
    xm = 0.5 * x[:-1] + 0.5 * x[1:]

    if not adjoint:
        # 1. element-level matrices, setup=2 (time-dependent linear)
        matrices, params = algmatrices(params, setup=2, adjoint=False)

        # 2. decide if k is constant or space/time dependent
        if not hasattr(params, "k") or getattr(params, "k") is None:
            # no k(x,t) field -> use constant kref
            matrices = algkmatrices(params, setup=2, matrices=matrices,
                                    k=params.kref, adjoint=False)
            kfun = 0
        else:
            # k is function or array -> we will update kref at each time step
            nx, nt = params.nx, params.nt
            matrices.storeBcal = np.zeros((2*nx+2, 2*nx+2, nt))
            matrices.storeBS   = np.zeros((2*nx+2, nx+1,   nt))
            matrices.storeM    = np.zeros((2*nx+2, 2*nx+2, nt))
            kfun = 1

        # 3. time stepping setup
        t = endpoints.tstart + params.deltat
        uhatm = np.zeros((nx + 1, nx + 1))   # û at each time level
        phatm = np.zeros((nx + 1, nx))       # p̂ at each time slab

        # initial condition: û(x, t_0)
        uhatm[:, 0] = source.IC(params.x)
        M, b = None, None

        # 4. time loop
        for k in range(nt):
            # (a) if k(x,t) is non-constant, update params.kref at t_k
            if kfun == 1:
                if callable(getattr(params, "k")):
                    # function of time
                    params.kref = params.k(t)
                else:
                    # matrix-valued k(x,t): take column k
                    k_arr = np.asarray(params.k)
                    # assume k_arr shape (nx, nt)
                    params.kref = k_arr[:, k]
                matrices = algkmatrices(params, setup=2,
                                        matrices=matrices,
                                        k=params.kref,
                                        adjoint=False)

            # (b) assemble matrices that do not depend on current u, but on k etc.
            matrices = bmatrices(params, matrices)
            if debug:
                print_simplenamespace(matrices)

            # (c) boundary conditions BC at time t_k
            BC = SimpleNamespace()
            # g(t) / g(k)
            if callable(getattr(source, "g")):
                BC.g = source.g(t)
            else:
                BC.g = source.g[k]

            # psi(t) / psi(k)
            if callable(getattr(source, "psi")):
                BC.psi = source.psi(t)
            else:
                BC.psi = source.psi[k]

            BC.bcu = source.bcu(t)
            BC.bcp = source.bcp(t)

            # (d) build RHS b at this time step
            b = derive_b(matrices, source, uhatm[:, k], x, t,
                         setup=2, endpoints=endpoints, params=params, BC=BC)
            
            # (e) solve for [û_{k+1}, p̂_k]
            Luu, Lup = matrices.Luu, matrices.Lup
            Lpu, Lpp = matrices.Lpu, matrices.Lpp
            uhat, phat, M = solveuhatphat(Luu, Lup, Lpu, Lpp, b, nx)
            
            uhatm[:, k+1] = uhat
            phatm[:, k]   = phat

            # (f) if k was time-dependent, store M,Bcal,BS for adjoint
            if kfun == 1:
                matrices.storeBcal[:, :, k] = matrices.Bcal
                matrices.storeBS[:, :, k]   = matrices.BcalS
                matrices.storeM[:, :, k]    = M

            # (g) advance time
            t += params.deltat

        # forward branch: usually û, p̂ covering all time steps
        return uhatm, phatm, M, b, matrices
    else:
        return "adjoint"
        # # 1) element-level matrices with adjoint flag
        # matrices, params = algmatrices(params, setup=2, adjoint=True)
        # matrices = algkmatrices(params, setup=2, matrices=matrices,
        #                         k=params.kref, adjoint=True)

        # # 2) initial Pi from ICm and ICh
        # #    firstuhat = source.ICm(params.x');
        # firstuhat = source.ICm(params.x)   # 注意：看你 Python 版 ICm 的接口
        # #    pi(k) = ∫ ICh(x) dx * n
        # pi = np.zeros(n)
        # for i in range(n):
        #     # 这里 integral(...) 你可以替换成自己的求积函数
        #     # MATLAB 用的是 integral(source.ICh, x(i), x(i+1))*n
        #     pi[i] = integral(source.ICh, x[i], x[i+1]) * n

        # Pi = solvePi(firstuhat, params.lambdae, matrices.G)

        # # 3) time grid backward
        # t = endpoints.tend - params.deltat

        # u = np.zeros((n, n))
        # p = np.zeros((n, n))
        # uhatm = np.zeros((n + 1, n))
        # phatm = np.zeros((n + 1, n))

        # # final state at t_end
        # u[:, -1] = source.ICm(xm)
        # p[:, -1] = source.ICh(xm)

        # # 4) backward in time
        # for k in range(n - 1):
        #     # (a) build Pi-dependent vectors at time t
        #     matrices = algPivectors(params, setup=2, matrices=matrices,
        #                             source=source,
        #                             endpoints=endpoints,
        #                             t=t,
        #                             Pi=Pi,
        #                             a=None,
        #                             hi=pi.T)   # or hi=pi depending on your shape

        #     # (b) build adjoint RHS (bu,bp)
        #     bu, bp, matrices = adjbvectors(params, matrices, params.kref, setup=2)

        #     # (c) BC at time t
        #     BC = SimpleNamespace()
        #     BC.g   = source.g(t)
        #     BC.psi = source.psi(t)
        #     BC.bcu = source.bcu(t)
        #     BC.bcp = source.bcp(t)

        #     # (d) solve adjoint system
        #     # MATLAB: [uhat,phat,M,b] = solveuhatphat(Luu,Lup,Lhm,Lpp,bu,bp,BC,n);
        #     uhat, phat, M_step, b_step = solveuhatphat(
        #         matrices.Luu, matrices.Lup,
        #         matrices.Lhm, matrices.Lpp,
        #         bu, bp, BC, n
        #     )

        #     Pi = solvePi(uhat, params.lambdae, matrices.G)

        #     # (e) recover physical u,p
        #     u[:, -1-k], p[:, -1-k] = solveup(uhat, phat, matrices)

        #     uhatm[:, -1-k] = uhat
        #     phatm[:, -1-k] = phat

        #     t -= params.deltat

        # # adjoint branch: 返回整个时间轨的 û, p̂ 以及最后一次 M,b
        # return uhatm, phatm, M_step, b_step, matrices

In [14]:
def approxPVEsol(params, source, endpoints, setup, nodes: int = None, adjoint: bool = False):
    """
    Approximate Poro-Visco-Elastic solver.

    Using sources to solve for u, p, and x for setup:
        1 = constant linear
        2 = nonconstant linear
        3 = constant nonlinear
        4 = nonconstant linear

    Parameters
    ----------
    params: dataclass
        parameter dataclass (physical + mesh)
    source: dataclass
        source dataclass (forcing, BC, IC)
    endpoints: 
        x/t intervals
    setup: int
        model selector
    nodes: int
        optional, number of spatial/time nodes (default: keep existing or 2560)
    adjoint: bool
        if True, compute adjoint system

    Returns
    ----------
    u, p: ndarray
        discrete solutions for displacement (u) and pressure (p) (or state variables)
    M, b: ndarray
        global FEM matrix (M) and right-hand-side vector from the FEM assembly
    matrices: dict or custom struct
        additional assembled matrices or bookkeeping data
    """
    
    ### changed!!! the way of setting defaults. 
    # ---------- [may debug here] ----------
    ### determine grid resolution, priority: function argument > external preset > default value
    if nodes is not None:
        params.nx = params.nt = nodes
    elif not (getattr(params, "nx", None) and getattr(params, "nt", None)):
        params.nx = params.nt = 2560
    # ---------- [may debug here] ----------

    ### replace source terms for adjoint computation
    if adjoint:
        return "adjoint"
        # for lhs, rhs in [("F", "z1"), ("S", "z2"), ("g", "z3"), ("psi", "z4")]:
        #     # a safer version of "source.lhs = source.rhs" to avoid None's
        #     if hasattr(source, rhs):
        #         setattr(source, lhs, getattr(source, rhs))
        #     else:
        #         print(f"Source has no attribute '{rhs}' for adjoint replacement.")

    ### build space and time grids
    params.x = np.linspace(endpoints.xstart, endpoints.xend, params.nx + 1)
    params.h = (endpoints.xend - endpoints.xstart) / params.nx
    params.deltat = (endpoints.tend - endpoints.tstart) / params.nt

    ### dispatch to the appropriate model solver according to the setup
    if setup == 1:
        if adjoint:
            return "adjoint"
            # u, p, mhat, hhat, M, b, matrices = constlinPVE(source, params, adjoint)
        else:
            u, p, mhat, hhat, M, b, matrices = constlinPVE(source, params)
    elif setup == 2:
        if adjoint:
            return "adjoint"
        else:
            u, p, M, b, matrices = linPVE(source, params, endpoints)
    # elif setup == 3:
    #     u, p, M, b, matrices = constnonlinPVE(source, params)
    # elif setup == 4:
    #     u, p, M, b, matrices = nonlinPVE(source, params, endpoints)
    #     u = u.T
    #     p = p.T
    else:
        raise ValueError("Enter 1 for constant linear, 2 for linear, 3 for constnat nonlinear, or 4 for nonlinear.")

    return u, p, M, b, matrices

### run_opt.m

In [15]:
params, endpoints, source = create_params_optimize()

t = np.linspace(endpoints.tstart, endpoints.tend, params.nt+1)
x = np.linspace(endpoints.xstart, endpoints.xend, params.nx+1)

### grid constants
params.deltat = (endpoints.tend - endpoints.tstart)/params.nt
params.deltax = (endpoints.xend - endpoints.xstart)/params.nx

### instead of the simple case params.kref = 1, can define params.k as 
# params.k    = lambda t: 4 * t + 1                           # a function of time
# params.k    = np.random.rand(params.nx, params.nt) * 5      # a function of x and t (need to put in a matrix)

### specific configs for each trial
scaleu, scalep  = 1, 1      # parameters to scale a constant in front of u-ud term and p-pd term in the loss
setup           = 2         # 1: constant linear; 2: linear; 3: constant non-linear; 4: non-linear

#### set sources

Find $\tilde{u}$ and $\tilde{p}$ which represent the solution with the control set to 0 and all other conditions set as desired.

In [20]:
source.F, source.S, source.g, source.psi, ud_fun, pd_fun, exactt = tc2source(params, endpoints.xend)

tend = t[1:]  # corresponds to MATLAB t(2:end), exclude t0 (IC) for optimization feedback
X_full, T_full = np.meshgrid(x, t)
X, T = np.meshgrid(x, tend)

### boundary & IC zeroed
source.bcu = lambda y: np.zeros_like(y)
source.bcp = lambda y: np.zeros_like(y)
source.IC  = lambda y: np.zeros_like(y)

u_tilde, p_tilde, M, b, matrices = approxPVEsol(params, source, endpoints, setup)

# source.F = np.zeros(params.nx + 1, params.nt + 1)
# source.F = lambda x, t: 0.0     # control set to 0

/var/folders/g4/hpzbn_117517262rrys1ftmr0000gn/T/ipykernel_89284/2915846986.py:27: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  scriptB = float(B @ Ai @ B.T)        # 1x1 -> scalar
/var/folders/g4/hpzbn_117517262rrys1ftmr0000gn/T/ipykernel_89284/2915846986.py:36: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  alpha = float(B @ Ai @ D)  # 1x1 → scalar
/var/folders/g4/hpzbn_117517262rrys1ftmr0000gn/T/ipykernel_89284/2419927618.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated Nu

In [26]:
threshold = 1e-13
mask = np.abs(u_tilde) < threshold
u_tilde[mask] = 0.0
print(u_tilde)

[[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.        ]
 [ 0.          0.10493352  0.19831187  0.26985542  0.31168822  0.31920508
   0.29157852  0.23184984  0.14659436  0.04519753 -0.06117824 -0.16082246
  -0.24276566 -0.29798702 -0.32040743 -0.30755869 -0.26085526 -0.18543853
  -0.08961081  0.0160786   0.11999479  0.21069803  0.27820318  0.31507886
   0.3172656 ]
 [ 0.          0.19830718  0.37477663  0.50998154  0.58903777  0.60324234
   0.55103156  0.43815313  0.27703343  0.08540953 -0.11562342 -0.30393446
  -0.45879313 -0.56315162 -0.60552148 -0.58123836 -0.49297545 -0.35044926
  -0.16934991  0.0303861   0.22677063  0.39818451  0.52575746  0.59544552
   0.59957701]
 [ 0.          0.26982435  0.50993453  0.69389777  0.80146232  0.82078686
   0.74974406  0.59615479